In [ ]:
!pip install dash plotly jupyter-dash
# run only once to install libraries

In [12]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

# ------------------------
# Load CSV safely
# ------------------------
def load_file(filename):
    return pd.read_csv(filename, encoding="utf-8-sig", on_bad_lines="skip")

results_df = load_file("results.csv")
limits_df  = load_file("limits.csv")

results_df.columns = results_df.columns.str.strip().str.lower()
limits_df.columns  = limits_df.columns.str.strip()

# ------------------------
# Prepare parameters
# ------------------------
parameters = limits_df["param_name"].unique()

# Merge limits
merged = results_df.merge(
    limits_df[["param_name", "Lower OK", "Upper OK"]],
    on="param_name",
    how="left"
)

merged["out_of_spec"] = (
    (merged["result"] < merged["Lower OK"]) |
    (merged["result"] > merged["Upper OK"])
)

# ------------------------
# JupyterDash App
# ------------------------
app = JupyterDash(__name__)

app.layout = html.Div(
    style={"padding": "10px"},
    children=[
        html.H2("Process Monitoring Interactive Dashboard"),

        dcc.Dropdown(
            id="param-select",
            options=[{"label": p, "value": p} for p in parameters],
            value=list(parameters),
            multi=True
        ),

        dcc.Graph(id="process-graph", style={"height": "85vh"})
    ]
)

# ------------------------
# Callback
# ------------------------
@app.callback(
    Output("process-graph", "figure"),
    Input("param-select", "value")
)
def update_graph(selected_params):

    if not selected_params:
        return go.Figure()

    fig = make_subplots(
        rows=len(selected_params),
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        subplot_titles=selected_params
    )

    for i, param in enumerate(selected_params, start=1):
        data = merged[merged["param_name"] == param].reset_index(drop=True)

        low  = data["Lower OK"].iloc[0]
        high = data["Upper OK"].iloc[0]

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data["result"],
                mode="lines+markers",
                name=f"{param} Result",
                customdata=data["uniquepart_id"],
                hovertemplate="Part: %{customdata}<br>Value: %{y}<extra></extra>"
            ),
            row=i, col=1
        )

        oos = data[data["out_of_spec"]]
        fig.add_trace(
            go.Scatter(
                x=oos.index,
                y=oos["result"],
                mode="markers",
                marker=dict(color="red", size=8),
                name="Out of Spec",
                showlegend=(i == 1),
                customdata=oos["uniquepart_id"],
                hovertemplate="Part: %{customdata}<br>OOS: %{y}<extra></extra>"
            ),
            row=i, col=1
        )

        fig.add_hline(y=high, line_dash="dash", line_color="red", row=i, col=1)
        fig.add_hline(y=low,  line_dash="dash", line_color="green", row=i, col=1)

        fig.add_hrect(
            y0=low,
            y1=high,
            fillcolor="green",
            opacity=0.1,
            line_width=0,
            row=i, col=1
        )

    fig.update_layout(
        height=300 * len(selected_params),
        hovermode="x unified",
        showlegend=True
    )

    return fig

# ------------------------
# RUN APP INSIDE NOTEBOOK
# ------------------------
app.run(mode="inline", debug=True)


C:\Users\soura\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py:644: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [11]:
pip install "dash[cloud]"

Note: you may need to restart the kernel to use updated packages.
